# Домашнее задание 3. Парсинг, Git и тестирование на Python

**Выполнила:** Смирнова Анастасия (код - 100499) 😀

**Цели задания:**

* Освоить базовые подходы к web-scraping с библиотеками `requests` и `BeautisulSoup`: навигация по страницам, извлечение HTML-элементов, парсинг.
* Научиться автоматизировать задачи с использованием библиотеки `schedule`.
* Попрактиковаться в использовании Git и оформлении проектов на GitHub.
* Написать и запустить простые юнит-тесты с использованием `pytest`.


В этом домашнем задании вы разработаете систему для автоматического сбора данных о книгах с сайта [Books to Scrape](http://books.toscrape.com). Нужно реализовать функции для парсинга всех страниц сайта, извлечения информации о книгах, автоматического ежедневного запуска задачи и сохранения результата.

Важной частью задания станет оформление проекта: вы создадите репозиторий на GitHub, оформите `README.md`, добавите артефакты (код, данные, отчеты) и напишете базовые тесты на `pytest`.



In [1]:
! pip install -q schedule pytest # установка библиотек, если ещё не

In [2]:
# Импортируем нужные библиотеки
import os
import time
import schedule

import pandas as pd
import requests
from bs4 import BeautifulSoup

## Задание 1. Сбор данных об одной книге (20 баллов)

В этом задании мы начнем подготовку скрипта для парсинга информации о книгах со страниц каталога сайта [Books to Scrape](https://books.toscrape.com/).

Для начала реализуйте функцию `get_book_data`, которая будет получать данные о книге с одной страницы (например, с [этой](http://books.toscrape.com/catalogue/a-light-in-the-attic_1000/index.html)). Соберите всю информацию, включая название, цену, рейтинг, количество в наличии, описание и дополнительные характеристики из таблицы Product Information. Результат достаточно вернуть в виде словаря.

**Не забывайте про соблюдение PEP-8** — помимо качественно написанного кода важно также документировать функции по стандарту:
* кратко описать, что она делает и для чего нужна;
* какие входные аргументы принимает, какого они типа и что означают по смыслу;
* аналогично описать возвращаемые значения.

*P. S. Состав, количество аргументов функции и тип возвращаемого значения можете менять как вам удобно. То, что написано ниже в шаблоне — лишь пример.*

In [3]:
# НАЧАЛО ВАШЕГО РЕШЕНИЯ
def get_book_data(book_url: str) -> pd.DataFrame:
    """
    Получает данные о книге со страницы и возвращает в виде DataFrame.

    Args:
        book_url (str): Ссылка на страницу с информацией о книге

    Returns:
        pd.DataFrame: DataFrame с одной строкой, содержащей информацию о книге.
                     Возвращает пустой DataFrame в случае ошибки.
    """
    try:
        # Отправляем запрос к странице и загружаем данные
        response = requests.get(book_url)

        if response.status_code != 200:
            error_log = (
                f"Ошибка при загрузке страницы {book_url}: {response.status_code}"
            )
            print(error_log)
            return pd.DataFrame()  # Возвращает пустой датафрейм в случае ошибки

        # Преобразуем HTML в объект BeautifulSoup
        soup = BeautifulSoup(response.text, "html.parser")

        # Создаем словарь для записи данных
        book_info = {}

        # Извлекаем данные с проверкой на существование элементов
        title_elem = soup.find("li", class_="active")
        title = title_elem.text if title_elem else "No title"

        breadcrumb_elem = soup.find("ul", class_="breadcrumb")
        genre = breadcrumb_elem.text.split()[2] if breadcrumb_elem else "No genre"

        rating_elem = soup.find('p', class_='star-rating')
        rating_text = rating_elem.get("class")[1] if rating_elem else "No rating"

        # Извлекаем описание
        description_div = soup.find('div', id='product_description')
        if description_div:
            description_p = description_div.find_next_sibling('p')
            description = (
                description_p.text.strip() if description_p else "No description"
            )
        else:
            description = "No description"

        product_information = soup.find(
            "table", class_="table table-striped"
        ).text.split()

        upc = product_information[0] if len(product_information) > 0 else ""
        product_type = (
            product_information[2][4:] if len(product_information) > 2 else ""
        )
        price_excl_tax = (
            product_information[5][6:] if len(product_information) > 5 else ""
        )
        price_incl_tax = (
            product_information[8][6:] if len(product_information) > 8 else ""
        )
        tax = product_information[9][5:] if len(product_information) > 9 else ""
        in_stock = (
            product_information[13][1:] if len(product_information) > 13 else ""
        )
        reviews_count = (
            product_information[18] if len(product_information) > 18 else ""
        )

        # Дополнительно преобразуем значения рейтинга в числовой формат
        rating_map = {"One": 1, "Two": 2, "Three": 3, "Four": 4, "Five": 5}
        numeric_rating = rating_map.get(rating_text)

        # Записываем данные в словарь
        book_info["title"] = title
        book_info["genre"] = genre
        book_info["rating"] = numeric_rating
        book_info["upc"] = upc
        book_info["product_type"] = product_type
        book_info["price_excl_tax"] = price_excl_tax
        book_info["price_incl_tax"] = price_incl_tax
        book_info["tax"] = tax
        book_info["in_stock"] = in_stock
        book_info["reviews_count"] = reviews_count
        book_info["description"] = description
        book_info["url"] = book_url

        # Преобразуем словарь в DataFrame
        book_df = pd.DataFrame([book_info])
        
        return book_df

    except Exception as e:
        print(f"Неожиданная ошибка при обработке {book_url}: {e}")
        return pd.DataFrame()
# КОНЕЦ ВАШЕГО РЕШЕНИЯ

In [4]:
# Используйте для самопроверки
book_url = 'https://books.toscrape.com/catalogue/soumission_998/index.html'
get_book_data(book_url)

,title,genre,rating,upc,product_type,price_excl_tax,price_incl_tax,tax,in_stock,reviews_count,description,url
0,Soumission,Fiction,1,UPC6957f44c3847a760,Books,50.10,50.10,0.00,20,0,"Dans une France assez proche de la nÃ´tre, un ...",https://books.toscrape.com/catalogue/soumissio...


## Задание 2. Сбор данных обо всех книгах (20 баллов)

Создайте функцию `scrape_books`, которая будет проходиться по всем страницам из каталога (вида `http://books.toscrape.com/catalogue/page-{N}.html`) и осуществлять парсинг всех страниц в цикле, используя ранее написанную `get_book_data`.

Добавьте аргумент-флаг, который будет отвечать за сохранение результата в файл: если он будет равен `True`, то информация сохранится в ту же папку в файл `books_data.txt`; иначе шаг сохранения будет пропущен.

**Также не забывайте про соблюдение PEP-8**

In [5]:
# НАЧАЛО ВАШЕГО РЕШЕНИЯ
def scrape_books(save_flag: bool = False, pages_num: int = None) -> pd.DataFrame:
    """
    Собирает данные о книгах со страниц каталога.

    Args:
        save_flag (bool): True, если нужно сохранить результаты анализа
                         в текстовый файл (по умолчанию False)
        
        pages_num (int, optional): Количество страниц для парсинга. 
                                  Если None - парсятся все страницы.

    Returns:
        pd.DataFrame: DataFrame с информацией о книгах
    """
    all_books = []
    page_num = 1

    while True:
        # Проверяем лимит страниц, если указан
        if pages_num is not None and page_num > pages_num:
            print(f"Достигнут установленный лимит в {pages_num} страниц(ы)")
            break

        # Формируем URL страницы
        if page_num == 1:
            page_url = "http://books.toscrape.com/catalogue/page-1.html"
        else:
            page_url = f"http://books.toscrape.com/catalogue/page-{page_num}.html"

        try:
            response = requests.get(page_url)

            # Проверяем, не достигнут ли конец каталога
            if response.status_code == 404:
                print("Достигнут конец каталога!")
                break

            if response.status_code != 200:
                print(
                    f"Возникла ошибка при обработке страницы №{page_num} "
                    f"(код: {response.status_code})"
                )
                break

            soup = BeautifulSoup(response.text, "html.parser")

            # Находим все книги на странице
            book_elements = soup.find_all('article', class_='product_pod')

            if not book_elements:
                print("На странице нет книг, завершаем...")
                break

            # Обрабатываем каждую книгу на странице
            books_on_page = 0
            for book in book_elements:
                try:
                    # Получаем относительную ссылку и преобразуем в абсолютную
                    relative_link = book.h3.a['href']
                    # Исправляем путь - убираем лишние "catalogue/" если есть
                    if relative_link.startswith('catalogue/'):
                        relative_link = relative_link[10:]  # Убираем 'catalogue/'
                    book_url = f"http://books.toscrape.com/catalogue/{relative_link}"

                    # Получаем данные о книге
                    book_data = get_book_data(book_url)

                    if not book_data.empty:
                        all_books.append(book_data)
                        books_on_page += 1

                except Exception as e:
                    continue

            print(
                f"Страница {page_num}: успешно обработано "
                f"{books_on_page}/{len(book_elements)} книг"
            )

            # Переходим к следующей странице
            page_num += 1
            time.sleep(0.5)  # Делаем паузу между страницами

        except requests.RequestException as e:
            print(f"Ошибка сети на странице {page_num}: {e}")
            break
        except Exception as e:
            print(f"Неожиданная ошибка на странице {page_num}: {e}")
            break

    # Объединяем все данные
    if all_books:
        result_df = pd.concat(all_books, ignore_index=True)
        print(f"\nПарсинг завершен! Собрано данных о {len(result_df)} книгах")

        # Сохраняем данные в текстовый файл
        if save_flag:
            try:
                # Текущая директория
                current_dir = os.getcwd()
                
                # Корень проекта
                project_root = os.path.dirname(current_dir)

                # Путь к файлу
                file_path = os.path.join(project_root, "artifacts", "books_data.txt")
                
                # Создаем папку artifacts в корне проекта
                artifacts_dir = os.path.join(project_root, "artifacts")
                os.makedirs(artifacts_dir, exist_ok=True)
                
                with open(file_path, 'w', encoding='utf-8') as file:
                    # Создаем шапку файла
                    file.write("-" * 70 + "\n")
                    file.write("Каталог книг с сайта Books to Scrape\n")
                    file.write("-" * 70 + "\n\n")
                    file.write(f"Всего книг в каталоге: {len(result_df)}\n")
                    file.write(f"Обработано страниц: {page_num - 1}\n")
                    if pages_num is not None:
                        file.write(f"Лимит страниц: {pages_num}\n")
                    file.write(
                        f"Дата создания отчета: "
                        f"{pd.Timestamp.now().strftime('%d.%m.%Y %H:%M')}\n\n"
                    )

                    # Данные по каждой книге
                    for index, row in result_df.iterrows():
                        file.write(f"   Книга #{index + 1}\n")
                        file.write(f"   Название: {row.get('title', 'N/A')}\n")
                        file.write(f"   Жанр: {row.get('genre', 'N/A')}\n")
                        file.write(f"   Рейтинг: {row.get('rating', 'N/A')}/5\n")
                        file.write(f"   UPC: {row.get('upc', 'N/A')}\n")
                        file.write(
                            f"   Цена (без налога): £{row.get('price_excl_tax', 'N/A')}\n"
                        )
                        file.write(
                            f"   Цена (с налогом): £{row.get('price_incl_tax', 'N/A')}\n"
                        )
                        file.write(f"   Налог: {row.get('tax', 'N/A')}\n")
                        file.write(f"   В наличии: {row.get('in_stock', 'N/A')} шт.\n")
                        file.write(f"   Отзывы: {row.get('reviews_count', 'N/A')}\n")
                        file.write(f"   Описание: {row.get('description', 'N/A')}\n")
                        file.write(f"   Ссылка: {row.get('url', 'N/A')}\n")
                        file.write("-" * 70 + "\n\n")

                print("Данные успешно сохранены в файл 'artifacts/books_data.txt'")

            except Exception as e:
                print(f"Ошибка при сохранении файла: {e}")

        return result_df
    else:
        print("Не удалось собрать данные ни об одной книге")
        return pd.DataFrame()
# КОНЕЦ ВАШЕГО РЕШЕНИЯ

In [6]:
# Проверка работоспособности функции для 5 страниц с сохранением данных в текстовый файл
res = scrape_books(save_flag=True, pages_num = 5) # Допишите ваши аргументы
print(type(res), len(res)) # и проверки
res.head(5) # Проверяем, что данные записались в датафрейм

Страница 1: успешно обработано 20/20 книг
Страница 2: успешно обработано 20/20 книг
Страница 3: успешно обработано 20/20 книг
Страница 4: успешно обработано 20/20 книг
Страница 5: успешно обработано 20/20 книг
Достигнут установленный лимит в 5 страниц(ы)

Парсинг завершен! Собрано данных о 100 книгах
Данные успешно сохранены в файл 'artifacts/books_data.txt'
<class 'pandas.core.frame.DataFrame'> 100


,title,genre,rating,upc,product_type,price_excl_tax,price_incl_tax,tax,in_stock,reviews_count,description,url
0,A Light in the Attic,Poetry,3,UPCa897fe39b1053632,Books,51.77,51.77,0.00,22,0,It's hard to imagine a world without A Light i...,http://books.toscrape.com/catalogue/a-light-in...
1,Tipping the Velvet,Historical,1,UPC90fa61229261140a,Books,53.74,53.74,0.00,20,0,"""Erotic and absorbing...Written with starling ...",http://books.toscrape.com/catalogue/tipping-th...
2,Soumission,Fiction,1,UPC6957f44c3847a760,Books,50.10,50.10,0.00,20,0,"Dans une France assez proche de la nÃ´tre, un ...",http://books.toscrape.com/catalogue/soumission...
3,Sharp Objects,Mystery,4,UPCe00eb4fd7b871a48,Books,47.82,47.82,0.00,20,0,"WICKED above her hipbone, GIRL across her hear...",http://books.toscrape.com/catalogue/sharp-obje...
4,Sapiens: A Brief History of Humankind,History,5,UPC4165285e1663650f,Books,54.23,54.23,0.00,20,0,From a renowned historian comes a groundbreaki...,http://books.toscrape.com/catalogue/sapiens-a-...


## Задание 3. Настройка регулярной выгрузки (10 баллов)

Настройте автоматический запуск функции сбора данных каждый день в 19:00.
Для автоматизации используйте библиотеку `schedule`. Функция должна запускаться в указанное время и сохранять обновленные данные в текстовый файл.



Бесконечный цикл должен обеспечивать постоянное ожидание времени для запуска задачи и выполнять ее по расписанию. Однако чтобы не перегружать систему, стоит подумать о том, чтобы выполнять проверку нужного времени не постоянно, а раз в какой-то промежуток. В этом вам может помочь `time.sleep(...)`.

Проверьте работоспособность кода локально на любом времени чч:мм.



In [7]:
# НАЧАЛО ВАШЕГО РЕШЕНИЯ
def setup_parsing_schedule(start_time: str, run_immediately: bool = False) -> None:
    """
    Настраивает и запускает ежедневный парсинг книг по расписанию.

    Args:
        start_time (str): Время запуска парсинга в формате 'HH:MM'
                         (например, '09:00')
        run_immediately (bool): Если True, запускает парсинг сразу при старте

    Returns:
        None
    """

    def parsing_task():
        """Внутренняя функция для выполнения парсинга"""
        current_time = time.strftime('%Y-%m-%d %H:%M:%S')
        print(f"Запуск парсинга с Books to Scrape в {current_time}")
        df = scrape_books(save_flag=True)

    # Настраиваем расписание
    schedule.every().day.at(start_time).do(parsing_task)
    print(f"Расписание настроено: ежедневный парсинг в {start_time}")

    # Запускаем сразу если нужно
    if run_immediately:
        print("Запуск немедленного парсинга...")
        parsing_task()

    # Бесконечный цикл проверки расписания
    print(f"Планировщик запущен. Начало парсинга в {start_time}")

    try:
        while True:
            schedule.run_pending()
            time.sleep(3600)  # Проверяем каждый час
    except KeyboardInterrupt:
        print("\nПланировщик остановлен пользователем")


# Запускаем ежедневно в 19:00
setup_parsing_schedule("19:00")
# КОНЕЦ ВАШЕГО РЕШЕНИЯ

Расписание настроено: ежедневный парсинг в 19:00
Планировщик запущен. Начало парсинга в 19:00

Планировщик остановлен пользователем


In [8]:
# Тестируем для всех страниц с немедленным запуском
setup_parsing_schedule("19:00", run_immediately = True)

Расписание настроено: ежедневный парсинг в 19:00
Запуск немедленного парсинга...
Запуск парсинга с Books to Scrape в 2025-10-24 11:05:39
Страница 1: успешно обработано 20/20 книг
Страница 2: успешно обработано 20/20 книг
Страница 3: успешно обработано 20/20 книг
Страница 4: успешно обработано 20/20 книг
Страница 5: успешно обработано 20/20 книг
Страница 6: успешно обработано 20/20 книг
Страница 7: успешно обработано 20/20 книг
Страница 8: успешно обработано 20/20 книг
Страница 9: успешно обработано 20/20 книг
Страница 10: успешно обработано 20/20 книг
Страница 11: успешно обработано 20/20 книг
Страница 12: успешно обработано 20/20 книг
Страница 13: успешно обработано 20/20 книг
Страница 14: успешно обработано 20/20 книг
Страница 15: успешно обработано 20/20 книг
Страница 16: успешно обработано 20/20 книг
Страница 17: успешно обработано 20/20 книг
Страница 18: успешно обработано 20/20 книг
Страница 19: успешно обработано 20/20 книг
Страница 20: успешно обработано 20/20 книг
Страница 21:

## Задание 4. Написание автотестов (15 баллов)

Создайте минимум три автотеста для ключевых функций парсинга — например, `get_book_data` и `scrape_books`. Идеи проверок (можете использовать свои):

* данные о книге возвращаются в виде словаря с нужными ключами;
* список ссылок или количество собранных книг соответствует ожиданиям;
* значения отдельных полей (например, `title`) корректны.

Оформите тесты в отдельном скрипте `tests/test_scraper.py`, используйте библиотеку `pytest`. Убедитесь, что тесты проходят успешно при запуске из терминала командой `pytest`.

Также выведите результат их выполнения в ячейке ниже.

**Не забывайте про соблюдение PEP-8**


In [10]:
# Ячейка для демонстрации работоспособности
!pytest ../tests/test_scraper.py -v -s

============================= test session starts ==============================
platform darwin -- Python 3.10.18, pytest-8.4.2, pluggy-1.6.0 -- /Users/steisha.s/opt/anaconda3/envs/books_parser_env/bin/python3.1
cachedir: .pytest_cache
rootdir: /Users/steisha.s/Desktop/Projects/books_parser
plugins: anyio-4.11.0
collected 6 items                                                              

../tests/test_scraper.py::TestBookParser::test_data_type_gbd PASSED
../tests/test_scraper.py::TestBookParser::test_columns_exist_gbd PASSED
../tests/test_scraper.py::TestBookParser::test_contents_gbd[0-Soumission] PASSED
../tests/test_scraper.py::TestBookParser::test_contents_gbd[1-Sharp Objects] PASSED
../tests/test_scraper.py::TestBookParser::test_contents_gbd[2-The Requiem Red] PASSED
Страницы:   0%|                                           | 0/5 [00:00<?, ?it/s]Страница 1: успешно обработано 20/20 книг
Достигнут установленный лимит в 1 страниц(ы)

Парсинг завершен! Собрано данных о 20 книгах


## Задание 5. Оформление проекта на GitHub и работа с Git (35 баллов)

В этом задании нужно воспользоваться системой контроля версий Git и платформой GitHub для хранения и управления своим проектом. **Ссылку на свой репозиторий пришлите в форме для сдачи ответа.**

### Пошаговая инструкция и задания

**1. Установите Git на свой компьютер.**

* Для Windows: [скачайте установщик](https://git-scm.com/downloads) и выполните установку.
* Для macOS:

  ```
  brew install git
  ```
* Для Linux:

  ```
  sudo apt update
  sudo apt install git
  ```

**2. Настройте имя пользователя и email.**

Это нужно для подписи ваших коммитов, сделайте в терминале через `git config ...`.

**3. Создайте аккаунт на GitHub**, если у вас его еще нет:
[https://github.com](https://github.com)

**4. Создайте новый репозиторий на GitHub:**

* Найдите кнопку **New repository**.
* Укажите название, краткое описание, выберите тип **Public** (чтобы мы могли проверить ДЗ).
* Не ставьте галочку Initialize this repository with a README.

**5. Создайте локальную папку с проектом.** Можно в терминале, можно через UI, это не имеет значения.

**6. Инициализируйте Git в этой папке.** Здесь уже придется воспользоваться некоторой командой в терминале.

**7. Привяжите локальный репозиторий к удаленному на GitHub.**

**8. Создайте ветку разработки.** По умолчанию вы будете находиться в ветке `main`, создайте и переключитесь на ветку `hw-books-parser`.

**9. Добавьте в проект следующие файлы и папки:**

* `scraper.py` — ваш основной скрипт для сбора данных.
* `README.md` — файл с кратким описанием проекта:

  * цель;
  * инструкции по запуску;
  * список используемых библиотек.
* `requirements.txt` — файл со списком зависимостей, необходимых для проекта (не присылайте все из глобального окружения, создайте изолированную виртуальную среду, добавьте в нее все нужное для проекта и получите список библиотек через `pip freeze`).
* `artifacts/` — папка с результатами парсинга (`books_data.txt` — полностью или его часть, если весь не поместится на GitHub).
* `notebooks/` — папка с заполненным ноутбуком `HW_03_python_ds_2025.ipynb` и запущенными ячейками с выводами на экран.
* `tests/` — папка с тестами на `pytest`, оформите их в формате скрипта(-ов) с расширением `.py`.
* `.gitignore` — стандартный файл, который позволит исключить временные файлы при добавлении в отслеживаемые (например, `__pycache__/`, `.DS_Store`, `*.pyc`, `venv/` и др.).


**10. Сделайте коммит.**

**11. Отправьте свою ветку на GitHub.**

**12. Создайте Pull Request:**

* Перейдите в репозиторий на GitHub.
* Нажмите кнопку **Compare & pull request**.
* Укажите, что было добавлено, и нажмите **Create pull request**.

**13. Выполните слияние Pull Request:**

* Убедитесь, что нет конфликтов.
* Нажмите **Merge pull request**, затем **Confirm merge**.

**14. Скачайте изменения из основной ветки локально.**



### Требования к итоговому репозиторию

* Файл `scraper.py` с рабочим кодом парсера.
* `README.md` с описанием проекта и инструкцией по запуску.
* Папка `artifacts/` с результатом сбора данных (`.txt` файл).
* Папка `tests/` с тестами на `pytest`.
* Папка `notebooks/` с заполненным ноутбуком `HW_03_python_ds_2025.ipynb`.
* Pull Request с комментарием из ветки `hw-books-parser` в ветку `main`.
* Примерная структура:

  ```
  books_scraper/
  ├── artifacts/
  │   └── books_data.txt
  ├── notebooks/
  │   └── HW_03_python_ds_2025.ipynb
  ├── scraper.py
  ├── README.md
  ├── tests/
  │   └── test_scraper.py
  ├── .gitignore
  └── requirements.txt
  ```